The following exercises are meant to be solved by gathering the bash commands incrimentally in two scripts, one for ex 1.* the other for ex 2.* 

### Ex 1

1\.a Make a new directory called `students` in your home. Download a csv file with the list of students of this lab from [here](https://www.dropbox.com/s/867rtx3az6e9gm8/LCP_22-23_students.csv) (use the `wget` command) and copy that to `students`. First check whether the file is already there

In [ ]:
mkdir students
wget -O data.csv "https://www.dropbox.com/scl/fi/bxv17nrbrl83vw6qrkiu9/LCP_22-23_students.csv?rlkey=47fakvatrtif3q3qw4q97p5b7&e=1"
cp data.csv /Users/enri/students/  
cd students 
ls 
#oppure 
find . -name "data.csv"

1\.b Make two new files, one containing the students belonging to PoD, the other to Physics.

In [ ]:
grep "PoD" data.csv > pod_students.csv 
grep "Physics" data.csv > physics_students.csv 
#to check if it correct:
head pod_students.csv
head physics_students.csv 

1\.c For each letter of the alphabet, count the number of students whose surname starts with that letter. 

In [ ]:
for letter in {A..Z}; do count=$(tail -n+2 data.csv | grep -i "^$letter" |wc -l); 
echo "$letter: $count"; 
done 
#tail -n+2 permette di non considerare l'intestazione nel conto, grep prende le 
#righe che iniziano con quella lettera (-i = case insensitive )
#wc -l te le conta , ed echo stampa il risultato

1\.d Find out which is the letter with most counts.

In [ ]:
max_count=0; max_letter=""; for letter in {A..Z}; do count=$(tail -n +2 data.csv | grep -i "^$letter" | wc -l); echo "$letter: $count"; if [ $count -gt $max_count ]; then max_count=$count; max_letter=$letter; fi; done   

1\.e Assume an obvious numbering of the students in the file (first line is 1, second line is 2, etc.), group students "modulo 18", i.e. 1,19,37,.. 2,20,38,.. etc. and put each group in a separate file  

In [ ]:
header=$(head -n1 data.csv) #prende solo la prima riga per i file, e la salva in una variabile chiamata header 
total=$(wc -l < data.csv) #conta quante righe(-l) ci sono nel file( inclusa l'intestazione) e le salva nella variabile total, il < invia il contenuto del file a wc -l
for group in {1..18}; do
echo "$header" > "group_${group}.csv";  #scrive l'intestazione nel file del gruppo
done;
for i in $(seq 2 $total); do #genera i numeri da 2 fino all'ultima riga( non 1, perche è l'intestazione)
line=$(sed -n "${i}p" data.csv); #prende solo la riga numero i dal file e la salva nella variabile line 
group=$(( (i-2) % 18 +1)); #(i-2) sempre perche sottre l'intestazione, %18 prende il resto della divisione e ci aggiunge 1, cosi che i gruppi diventino 1, 18 ecc e non 0, 17 
outfile="group_${group}.csv"; #costruisce un file per ogni gruppo: group_1.csv 
echo "$line" >> "$outfile"; #aggiunge la linea alla fine del file ( è tipo un append) senza cancellare il contenuto precedente 
done 

#altrimenti invece du creare il file outfile, puoi anche fare direttamente echo "$line" >> "group_${group}.csv" come vuoi 

### Ex 2

2.a Make a copy of the file `data.csv` removing the metadata and the commas between numbers; call it `data.txt`

In [ ]:
sed '/^#/d; s/,//g' data.txt > data_clean.txt
#le prime virgolette prendono tutti le righe che iniziano per # e con d le elimina
#dopo di che le seconde virgolette fanno s/vecchio/nuovo/g, in questo caso sostituiscono le virgolette con il niente
#e poi scrive il file in output in cui metterle 

2\.b How many even numbers are there?

In [ ]:
count=0
for number in $(cat data.txt);
do if [ $((num % 2)) -eq 0 ]; 
then count=$((count +1));
fi;
done;
echo "Numero di numeri pari: $count"
#cat fa un sacco di cose, tra cui leggere le cose nei file come in questi caso. 

2\.c Distinguish the entries on the basis of `sqrt(X^2 + Y^2 + Z^2)` is greater or smaller than `100*sqrt(3)/2`. Count the entries of each of the two groups 

In [ ]:
count_gt=0; count_le=0; soglia=$(echo "100*sqrt(3)/2" |bc -l); while read x y z rest; do r=$(echo "sqrt($x*$x+$y*$y+$z*$z)" | bc -l); if (( $(echo "$r > $soglia" | bc -l) )); then count_gt=$((count_gt +1)); else count_le=$((count_le +1)); fi; done < data.txt; echo "r > 100*sqrt(3)/2: $count_gt"; echo "r <= 100*sqrt(3)/2: $count_le"

#bc -l è letteralmente una calcolatrice da terminale
#da uno strano errore: Parse error: bad expression
   # <stdin>:1


#Parse error: bad token
    #<stdin>:1

#per questo l'ho modificato in questo modo, per cui mi legge e prende solo le righe non vuote 
count_gt=0; count_le=0; soglia=$(echo "100*sqrt(3)/2" |bc -l); while read x y z rest; do if [[ -z "$x" || -z "$y" || -z "$z" ]]; then continue; fi; r=$(echo "sqrt($x*$x+$y*$y+$z*$z)" | bc -l); if (( $(echo "$r > $soglia" | bc -l) )); then count_gt=$((count_gt +1)); else count_le=$((count_le +1)); fi; done < data.txt; echo "r > 100*sqrt(3)/2: $count_gt"; echo "r <= 100*sqrt(3)/2: $count_le"


2\.d Make `n` copies of data.txt (with `n` an input parameter of the script), where the i-th copy has all the numbers divided by i (with `1<=i<=n`).

In [ ]:
n=1
for ((i=1; i<=n; i++));
do outfile="copy_${i}.txt";
> "$outfile";
while read -r line; do
result="";
for num in $line; do
divnum=$(echo "$num / $i" | bc -l);
result="$result $divnum";
done;
echo "$result">>"$outfile";
done < data.txt;
done;
echo "Created $n copies of data.txt, each divided by 1..$n"

#la prima riga è un ciclo numerico per cui, inizializza i= 1, continua finche i non è minore di n, e incrementa di 1 dopo ogni ciclo i++, equivalenge a i=i+1
#while read -r legge riga per riga il file data.txt, -r serve per leggere le righe letteralmente, senza interpretare \
#DEVI STARE SU "BASH" scrivi su terminale semplicemente bash e andrà, se vuoi uscire fai exit 
